<a href="https://colab.research.google.com/github/ltabares/ClassificationCNNPneumonnia/blob/main/ClasificacionCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!git clone https://github.com/ltabares/ClassificationCNNPneumonnia.git

fatal: destination path 'ClassificationCNNPneumonnia' already exists and is not an empty directory.


# Nueva sección

# **Referencies**


1.   To build a CNN using upload images

      https://www.tensorflow.org/tutorials/load_data/images

2.   Equalization Histogram

    https://towardsdatascience.com/image-augmentation-for-deep-learning-using-keras-and-histogram-equalization-9329f6ae5085
    https://www.youtube.com/watch?v=mSbJ1Rqhze0




In [ ]:
#Functions
#Contrast adjustment using ecualization histogram

def contraste_img_eh(cat, path_data, type_image, size_image, path_data_he):
  #Args:
    #cat: category
    #path_data: path where the original data are saved
    #type_image: type of image (in this case jpeg)
    #size_image:size of image
    #path_data_he: path wherethe ecualizated image data are saved
    os.chdir(path_data + cat + '/')
    file_name_image = [file for file in glob.glob('*.' + type_image)]
    image = [load_img(path_data + cat + '/'+ x, target_size = (size_image,size_image)) for x in file_name_image]
    image_eh = list(map(ImageOps.equalize, image))
    print(file_name_image)
    for i in range(0, len(image_eh)):
      image_eh[i].save(path_data_he + cat + "/" + file_name_image[i])


In [ ]:
import numpy as np
import pandas as pd
import os, glob

import tensorflow as tf
#from tensorflow.keras.preprocessing.image import load_img
from keras_preprocessing.image import load_img

from PIL import Image, ImageOps 

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix

from google.colab import drive
from google.colab.patches import cv2_imshow # to show images in Google Colab
# To mount Google Drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#Configuration

#Paths
path_data_train = '/gdrive/MyDrive/Colab Notebooks/CNN Pneumonia/train/'
path_data_train_he = '/gdrive/MyDrive/Colab Notebooks/CNN Pneumonia/train_he/'

path_data_test = '/gdrive/MyDrive/Colab Notebooks/CNN Pneumonia/test/'
path_data_test_he = '/gdrive/MyDrive/Colab Notebooks/CNN Pneumonia/test_he/'

#Categories
cat = ['NORMAL', 'PNEUMONIA']

#Type of images
type_image = 'jpeg'

#If we want to apply ecualization hisogram over the original data and save them in a new folder 
#This is made in the fisrt running
eh = False

#Size of the images
size_image = 224

#Size of the batches
batch_size = 15

In [ ]:
#Amount of origianl data
for cat_ in cat:
  print('Datos ' + cat_ + ' : ' + str(len(os.listdir(path_data_train + cat_))))

Datos NORMAL : 1341
Datos PNEUMONIA : 3875


In [ ]:
if eh:
  contraste_img_eh('NORMAL', path_data_train, type_image, size_image, path_data_train_he)
  contraste_img_eh('PNEUMONIA', path_data_train, type_image, size_image, path_data_train_he)
  contraste_img_eh('NORMAL', path_data_test, type_image, size_image, path_data_test_he)
  contraste_img_eh('PNEUMONIA', path_data_test, type_image, size_image, path_data_test_he)

  #Amount of new ecualizated images. 
  #testing we haven't lost any image
  for cat_ in cat:
    print('Datos ' + cat_ + ' : ' + str(len(os.listdir(path_data_train_he + cat_))))

In [ ]:
#Get the data from the directory and creat a BatchDataset object for trainnig set
train_data = tf.keras.utils.image_dataset_from_directory(
  path_data_train_he,
  labels = 'inferred',
  class_names= ['NORMAL', 'PNEUMONIA'],
  color_mode = 'grayscale',
  shuffle= True, 
  seed= 123,
  image_size= (size_image, size_image),
  batch_size= batch_size)


Found 5216 files belonging to 2 classes.


In [ ]:
#Get the data from the directory and creat a BatchDataset object for test set
test_data = tf.keras.utils.image_dataset_from_directory(
  path_data_test_he,
  labels = 'inferred',
  class_names= ['NORMAL', 'PNEUMONIA'],
  color_mode = 'grayscale',
  shuffle= True,
  seed= 123,
  image_size= (size_image, size_image),
  batch_size= batch_size)


y_test = np.concatenate([y for x, y in test_data], axis=0)
x_test = np.concatenate([x for x, y in test_data], axis=0)  

Found 624 files belonging to 2 classes.


In [ ]:
for image_batch, labels_batch in train_data:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(15, 224, 224, 1)
(15,)


In [ ]:
#Processing data
#1)Rescaling the images--> values[0,1]
#Get the max value of one image
max_value = tf.reduce_max(np.array(load_img(path_data_train_he + 'NORMAL/'+ 'NORMAL2-IM-1423-0001.jpeg', target_size = (size_image,size_image)), dtype= 'float32'))
print('El valor máximo de pixel es :', max_value.numpy().max())

resc = tf.keras.Sequential([tf.keras.layers.Rescaling(scale= 1./max_value.numpy().max(), offset= 0.0, name= 'rescaladoImagen')])


El valor máximo de pixel es : 255.0


In [ ]:
#2)Zoom in 
zoom_in = tf.keras.Sequential([tf.keras.layers.RandomZoom(height_factor= [-0.1, -0.2], width_factor= [-0.1, -0.2], name= 'zoomInImagen')])


#3)Rotation 
rotation = tf.keras.Sequential(tf.keras.layers.RandomRotation([-0.05, 0.05], fill_mode='nearest', name = 'rotationImage'))

In [ ]:
#CNN
filter_conv1 = 16
size_filter1 = (5,5)
size_pool1= (2,2)

filter_conv2 = 32
size_filter2 = (5,5)
size_pool2= (2,2)

epochs = 5

#NN
neuron_dense1 = 120
drop_out = 0.5

num_cat = len(cat)

In [ ]:
modelo = tf.keras.Sequential([
    resc, 
    zoom_in,
    rotation,
    tf.keras.layers.Conv2D(filter_conv1, size_filter1, padding = 'same', activation= 'relu', name = 'conv1'), 
    tf.keras.layers.MaxPooling2D(pool_size= size_pool1, name = 'pooling1'),
    tf.keras.layers.Conv2D(filter_conv2, size_filter2, padding = 'same', activation= 'relu', name = 'conv2'), 
    tf.keras.layers.MaxPooling2D(pool_size= size_pool2, name = 'pooling2'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(neuron_dense1, activation = 'relu', name = 'FC1'),
    tf.keras.layers.Dropout(drop_out), 
    tf.keras.layers.Dense(num_cat, activation='softmax', name = 'output')
])

In [ ]:
modelo.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
modelo.fit(
  train_data,
  validation_data = test_data,
  epochs = epochs
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


348/348 [==============================] - 539s 2s/step - loss: 0.3732 - accuracy: 0.8422 - val_loss: 0.4132 - val_accuracy: 0.8077
Epoch 2/5
348/348 [==============================] - 219s 629ms/step - loss: 0.2457 - accuracy: 0.8959 - val_loss: 0.6160 - val_accuracy: 0.7885
Epoch 3/5
348/348 [==============================] - 219s 629ms/step - loss: 0.2146 - accuracy: 0.9168 - val_loss: 0.4464 - val_accuracy: 0.7821
Epoch 4/5
348/348 [==============================] - 218s 627ms/step - loss: 0.1912 - accuracy: 0.9225 - val_loss: 0.4244 - val_accuracy: 0.7901
Epoch 5/5
348/348 [==============================] - 218s 626ms/step - loss: 0.1715 - accuracy: 0.9346 - val_loss: 0.4544 - val_accuracy: 0.8510
